# Урок 3. Dataset, Dataloader, BatchNorm, Dropout, Оптимизация

- Создать Dataset для загрузки данных (sklearn.datasets.fetch_california_housing)
- Обернуть его в Dataloader
- Написать архитектуру сети, которая предсказывает стоимость недвижимости. Сеть должна включать BatchNorm слои и Dropout (или НЕ включать, но нужно обосновать)
- Сравните сходимость Adam, RMSProp и SGD, сделайте вывод по качеству работы модели train-test разделение нужно сделать с помощью sklearn random_state=13, test_size = 0.25




In [1]:
import torch
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
from sklearn.metrics import r2_score
from sklearn import datasets
from sklearn.model_selection import train_test_split
from torch import optim
import warnings
import math
from random import shuffle
from tools import paralell_execution, ar_split_eq_cpu
warnings.filterwarnings('ignore')


class CaliforniaDataset(torch.utils.data.Dataset):
   
    def __init__(self, init_dataset, init_target, transform=None):
        self._base_dataset = torch.from_numpy(init_dataset).type(torch.float)
        self._base_targets = torch.from_numpy(init_target).type(torch.float)
        self.transform = transform

    def __len__(self):
        return len(self._base_dataset)

    def __getitem__(self, idx):
        features = self._base_dataset[idx]
        target = self._base_targets[idx]

        if self.transform is not None:
            features = self.transform(features)
      
        return features, target


class Perceptron(nn.Module):
    def __init__(self, input_dim, output_dim, activation="relu"):
        super().__init__()
        self.fc = nn.Linear(input_dim, output_dim)
        self.activation = activation
        
    def forward(self, x):
        x = self.fc(x)
        if self.activation == "relu":
            return F.relu(x)
        if self.activation == "sigmoid":
            return F.sigmoid(x)
        if self.activation == "leaky_relu":
            return F.leaky_relu(x)
        raise RuntimeError
        

class FeedForward(nn.Module):
    def __init__(self, input_dim, hidden_dim):
        super().__init__()
        self.fc1 = Perceptron(input_dim, hidden_dim, 'leaky_relu')
        self.bn = nn.BatchNorm1d(hidden_dim)
        self.dp = nn.Dropout(0.15)
        self.fc2 = Perceptron(hidden_dim, 1, 'leaky_relu')
        
    def forward(self, x):
        x = self.fc1(x)
        x = self.dp(x)
        x = self.bn(x)
        x = self.fc2(x)

        return x

In [2]:
# комбинации параметров нейросети 
params = []

for optim_func in [optim.Adam, optim.RMSprop, optim.SGD]:
    for lr in [1e-3, 5e-3]: # 1e-3, 5e-3, 1e-2
        for epochs in [15, 55]:
            params.append({
                'optim_func': optim_func, 
                'lr': lr, 
                'epochs': epochs, 
            })

shuffle(params)
# params = params[:1]
print(f'Всего комбинаций параметров: {len(params)}')

def train(params):
    ret = []
    for param in params:
        data = datasets.fetch_california_housing()
    
        X_train, X_test, y_train, y_test = train_test_split(data['data'], data['target'], test_size=0.2, random_state=42)
        
        train_dataset = CaliforniaDataset(X_train, y_train)
        train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=150, shuffle=False)
        
        test_dataset = CaliforniaDataset(X_test, y_test)
        test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=10, shuffle=False)
        
        net = FeedForward(8, 8)
        
        criterion = nn.MSELoss()
    
        optimizer = param['optim_func'](net.parameters(), lr=param['lr'])
        for epoch in range(param['epochs']):  
            running_loss, running_items, r2 = 0.0, 0.0, 0.0
    
            for i, data in enumerate(train_loader):
                fets, target = data[0], data[1]
                
                optimizer.zero_grad() # обнуляем градиент
    
                outputs = net(fets)
                loss = criterion(outputs, target)
                loss.backward()
                optimizer.step()
    
                running_loss += loss.item()
                running_items += len(target)
    
                predict = outputs.data.numpy()
                tr_target = target.view(target.shape[0], 1).numpy()
                r2 += r2_score(tr_target, predict)
                running_loss, running_items, r2 = 0.0, 0.0, 0.0

        # вычисляем точность
        net.eval()
        data = list(test_loader)[0]
        test_outputs = net(data[0])
        test_predict = test_outputs.data.numpy()
        te_target = data[1].view(data[1].shape[0], 1)
        r2_result = r2_score(te_target, test_predict)
        net.train()
        
        ret.append({
            'r2': r2_result,
            'param': param,
            'net': net
        })
    return ret

args_parallel = []
for param_chunk in ar_split_eq_cpu(params):
    args_parallel.append(param_chunk)

# запускаем процесс параллельных вычислений
res = paralell_execution(func=train,
                         arg=args_parallel,
                         method='multiprocessing')

best_r2 = None # точность лучшей модели
best_param = {} # параметры лучшей модели
best_net = None # лучшая модель
for chunk in res: # объединяем результаты со всех ядер
    for row in chunk:
        if best_r2 is None or abs(row['r2']) < abs(best_r2):
            best_r2 = row['r2']
            best_param = row['param']
            best_net = row['net']

print(f'Лучшая r2_score: {round(best_r2, 3)} при параметрах {best_param}')


Всего комбинаций параметров: 12
Лучшая r2_score: -0.021 при параметрах {'optim_func': <class 'torch.optim.rmsprop.RMSprop'>, 'lr': 0.005, 'epochs': 55}
